# Credit Card Customer Churn Prediction using Neural Networks

This notebook demonstrates how to build a deep learning model to predict whether a bank customer will leave (churn) or stay.

## What is Customer Churn?

Customer churn is when customers stop doing business with a company. For banks, predicting which customers might leave helps them take preventive actions to retain valuable customers.

## Project Overview

We'll follow these steps:
1. **Load and explore** the customer data
2. **Clean and prepare** the data for machine learning
3. **Build a neural network** to predict churn
4. **Train and evaluate** the model's performance
5. **Experiment** with different model configurations

Let's get started! 

## Step 1: Import Required Libraries

We need two essential Python libraries:
- **pandas**: For working with data in table format (like Excel)
- **numpy**: For mathematical operations and working with arrays

In [1]:
import pandas as pd
import numpy as np

## Step 2: Load the Dataset

We load the customer data from a CSV file. The `.head()` function shows us the first 5 rows so we can see what the data looks like.

**Dataset contains**: Customer information like credit score, geography, gender, age, account balance, and whether they left the bank (Exited = 1) or stayed (Exited = 0).

In [2]:
df = pd.read_csv("./datasets/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Step 3: Explore the Data

Before building our model, we need to understand what we're working with:
- **Shape**: How many rows (customers) and columns (features) do we have?
- **Info**: What types of data are in each column?
- **Duplicates**: Are there any repeated rows we should remove?

In [3]:
# Check the dimensions: (rows, columns)
df.shape

(10000, 14)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
df.duplicated().sum()

np.int64(0)

### Understanding Key Features

Let's examine some important columns:
- **Exited**: Our target variable (0 = stayed, 1 = left)
- **Geography**: Which country the customer is from
- **Gender**: Customer's gender

In [6]:
# Distribution of customers by country
df['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [7]:
# Distribution by gender
df['Gender'].value_counts()

Gender
Male      5457
Female    4543
Name: count, dtype: int64

## Step 4: Data Cleaning

We remove columns that won't help predict churn:
- **RowNumber**: Just an index, not useful
- **CustomerId**: A unique identifier, doesn't help with prediction
- **Surname**: Customer names don't indicate churn behavior

In [8]:
# Remove unnecessary columns
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [9]:
# Verify the columns were removed
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Step 5: Encoding Categorical Variables

Neural networks work with numbers, not text. We convert categorical features (Geography, Gender) into numerical format using **one-hot encoding**.

**Example**: Instead of "France", "Germany", "Spain", we create separate columns:
- Geography_Germany (1 or 0)
- Geography_Spain (1 or 0)

`drop_first=True` avoids redundancy (if both are 0, it must be France).

In [10]:
# Convert text categories to numbers
df = pd.get_dummies(df, columns=["Geography", "Gender"], drop_first=True)

In [11]:
# Check the new shape and structure
print(df.shape)
df.head()

(10000, 12)


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,False,False,False
1,608,41,1,83807.86,1,0,1,112542.58,0,False,True,False
2,502,42,8,159660.80,3,1,0,113931.57,1,False,False,False
3,699,39,1,0.00,2,0,0,93826.63,0,False,False,False
4,850,43,2,125510.82,1,1,1,79084.10,0,False,True,False


## Step 6: Prepare Data for Training

### Split into Features (X) and Target (y)

- **X**: All the input features (credit score, age, balance, etc.) - what we use to make predictions
- **y**: The target variable (Exited) - what we're trying to predict

### Train/Test Split

We divide our data into two sets:
- **Training set (80%)**: Used to teach the model
- **Test set (20%)**: Used to evaluate how well the model performs on new, unseen data

This is crucial to ensure our model generalizes well!

In [12]:
# Separate features from target
X = df.drop("Exited", axis=1)  # Features
y = df["Exited"]  # Target


# Split into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [13]:
# Verify the split dimensions
print("Training features:", X_train.shape)
print("Test features:", X_test.shape)
print("Training targets:", y_train.shape)
print("Test targets:", y_test.shape)

Training features: (8000, 11)
Test features: (2000, 11)
Training targets: (8000,)
Test targets: (2000,)


## Step 7: Feature Scaling

### Why Scale?

Our features have very different ranges:
- Age: 18-92
- Balance: 0-250,000
- NumOfProducts: 1-4

Neural networks learn better when all features are on a similar scale (typically mean=0, std=1).

**StandardScaler** transforms features to have:
- Mean (average) = 0
- Standard deviation = 1

⚠️ **Important**: We fit the scaler ONLY on training data to avoid data leakage!

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit and transform training data
X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)
# Only transform test data (using training statistics)

### Understanding fit_transform() vs transform()

#### 1. scaler.fit_transform(X_train)
This does two things at once:

Fit: It calculates the necessary parameters (e.g., mean and standard deviation) from your training data.
Transform: It applies the scaling formula to the training data using those calculated parameters.

#### 2. scaler.transform(X_test)
This only does one thing:

Transform: It applies the scaling to your test data using the same mean and standard deviation that were calculated from the training set.

## Step 8: Build the Neural Network

### What is a Neural Network?

A neural network is a machine learning model inspired by the human brain. It consists of:
- **Layers**: Groups of neurons (nodes) that process information
- **Neurons**: Individual units that perform calculations
- **Weights**: Numbers that get adjusted during training to improve predictions
- **Activation Functions**: Add non-linearity to help the model learn complex patterns

We'll use **TensorFlow/Keras**, a popular deep learning framework.

In [15]:
# Preview the scaled data
X_train_scaled

array([[-0.23082038, -0.94449979, -0.70174202, ...,  1.71490137,
        -0.57273139,  0.91509065],
       [-0.25150912, -0.94449979, -0.35520275, ..., -0.58312392,
        -0.57273139, -1.09278791],
       [-0.3963303 ,  0.77498705,  0.33787579, ...,  1.71490137,
        -0.57273139, -1.09278791],
       ...,
       [ 0.22433188,  0.58393295,  1.3774936 , ..., -0.58312392,
        -0.57273139, -1.09278791],
       [ 0.13123255,  0.01077067,  1.03095433, ..., -0.58312392,
        -0.57273139, -1.09278791],
       [ 1.1656695 ,  0.29735181,  0.33787579, ...,  1.71490137,
        -0.57273139,  0.91509065]], shape=(8000, 11))

In [16]:
# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

### Model Architecture

Our neural network has 3 layers:

1. **Input Layer** (11 nodes)
   - Takes in all 11 features
   - Uses ReLU activation (keeps positive values, zeros out negative ones)

2. **Hidden Layer** (11 nodes)
   - Learns complex patterns from the input
   - Also uses ReLU activation

3. **Output Layer** (1 node)
   - Produces final prediction: probability of churn
   - Should use sigmoid activation for binary classification (0 to 1)

💡 **Tip**: You can experiment with different architectures using the interactive widget below!

In [17]:
# Create a sequential model (layers stacked one after another)
model = Sequential()

# Input layer: 11 features → 11 neurons 
model.add(Dense(11, activation='relu', input_dim=11))

# Hidden layer: 11 neurons → 11 neurons
model.add(Dense(11, activation='relu'))

# Output layer: 11 neurons → 1 output (churn probability)
model.add(Dense(1, activation='sigmoid'))  # sigmoid for binary classification

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
# Display the model architecture and parameter count
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 11)             │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 11)             │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 276 (1.08 KB)

 Trainable params: 276 (1.08 KB)

 Non-trainable params: 0 (0.00 B)

## Step 9: Compile the Model

Before training, we need to configure the learning process:

- **Loss Function** (`binary_crossentropy`): Measures how wrong our predictions are (lower is better)
- **Optimizer** (`Adam`): Algorithm that adjusts weights to minimize loss (Adam is popular and effective)
- **Metrics** (`accuracy`): Track the percentage of correct predictions

In [19]:
# Configure the model for training
model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

## Step 10: Train the Model

### Training Process

The model learns by:
1. Making predictions on training data
2. Calculating how wrong it is (loss)
3. Adjusting weights to reduce the error
4. Repeating this process for multiple **epochs**

**Epoch**: One complete pass through all training data

**Validation Split (20%)**: We hold out 20% of training data to monitor performance during training and detect overfitting.

This may take a few minutes...

In [20]:
# Train the model for 100 epochs
history = model.fit(X_train_scaled,y_train,epochs=10)
# validation_split=0.2 means 20% of training data is used for validation 

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 318us/step - accuracy: 0.6754 - loss: 0.6006
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 265us/step - accuracy: 0.8030 - loss: 0.4557
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 264us/step - accuracy: 0.8106 - loss: 0.4330
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 264us/step - accuracy: 0.8161 - loss: 0.4209
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 267us/step - accuracy: 0.8209 - loss: 0.4079
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 266us/step - accuracy: 0.8346 - loss: 0.3916
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 265us/step - accuracy: 0.8431 - loss: 0.3740
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 263us/step - accuracy: 0.8506 - loss: 0.3628
Epoch 9/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 266us/step - accuracy: 0.8540 - loss: 0.3562
Epoch 10/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 262us/step - accuracy: 0.8551 - loss: 0.3529


In [21]:
# View weights and biases of the first layer
model.layers[0].get_weights()  # [weights_matrix, biases_array]

[array([[ 0.22044222, -0.07877811, -0.25194228,  0.05746334, -0.11340291,
          0.3972954 , -0.27651992,  0.11426117,  0.05172461, -0.31351998,
          0.0141973 ],
        [-0.73577213, -0.08068595, -0.15103553, -0.14402026, -0.54435337,
          0.14098826,  0.04091752,  0.5942597 , -0.7858449 ,  0.25799996,
          0.15511133],
        [-0.2737168 , -0.15565991,  0.16325729,  0.03532368, -0.23384626,
         -0.48629174, -0.20390128,  0.17222956,  0.36446875,  0.38515642,
          0.25238124],
        [-0.6047332 , -0.36385477,  0.31428486,  0.16870014,  0.34191942,
         -0.4473756 ,  0.22193693, -0.18841882,  0.4884162 , -0.01272488,
          0.51807225],
        [ 0.14253478, -1.2412618 , -0.76019996,  0.9597365 , -0.1774616 ,
          0.24613245,  0.7002163 , -0.45678684, -0.07683929,  0.10903496,
          0.6084481 ],
        [ 0.07906118, -0.04134243,  0.13130988,  0.00499383,  0.00523994,
          0.06501654,  0.08572985,  0.15941481, -0.13213547,  0.4211234

In [22]:
# View weights and biases of the second layer
model.layers[1].get_weights()

[array([[-8.31993222e-02,  2.76601702e-01, -1.75848976e-01,
          1.99396908e-01,  3.86051893e-01, -3.27143178e-04,
          6.15964770e-01,  4.60645199e-01,  1.48802236e-01,
          3.55755873e-02, -1.71587467e-02],
        [-7.63611615e-01, -3.78928214e-01, -2.36432418e-01,
          1.23589583e-01,  5.51377535e-02, -7.26552904e-01,
         -2.57313132e-01, -1.03555717e-01,  4.49744582e-01,
         -2.37567291e-01,  4.30220723e-01],
        [-4.04496849e-01, -3.57823610e-01,  4.45471071e-02,
          4.05537158e-01,  3.48446190e-01, -3.22146922e-01,
         -3.53169352e-01,  2.88264245e-01,  5.33310711e-01,
          4.39826161e-01, -1.55485213e-01],
        [-1.93344235e-01, -1.39125928e-01,  4.73820686e-01,
          1.79349840e-01,  9.25576910e-02,  1.61934316e-01,
          1.28446996e-01, -3.74523729e-01,  3.48647416e-01,
          2.31066376e-01,  3.58182192e-01],
        [ 7.77090415e-02,  2.13232458e-01, -3.05554211e-01,
         -3.45966548e-01,  1.24036940e-02, -

## Step 11: Make Predictions on Test Data

Now let's see how well our trained model performs on new, unseen data!

1. **Predict probabilities**: Get the model's confidence that each customer will churn (0 to 1)
2. **Convert to binary predictions**: If probability > 0.5, predict churn (1), otherwise predict stay (0)

In [23]:
# Get predicted probabilities for test set
y_log = model.predict(X_test_scaled)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step


In [24]:
# Convert probabilities to binary predictions (0 or 1)
y_pred = np.where(y_log > 0.5, 1, 0)  # Threshold at 0.5

## Step 12: Evaluate Model Performance

**Accuracy** = (Correct Predictions) / (Total Predictions)

This tells us what percentage of customers we correctly classified as churning or staying.

Typical results: 80-86% accuracy

In [25]:
from sklearn.metrics import accuracy_score

# Calculate accuracy on test settest_accuracy
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

Test Accuracy: 0.8575 (85.75%)


---

## Interactive Model Trainer

Now it's your turn to play data scientist! Use the interactive controls below to experiment with different neural network configurations.

### How to Use:

1. Move the sliders to set your desired configuration
2. Click the **Train Model** button
3. Watch the training progress
4. Compare the accuracy results with different settings

### Adjust Model Architecture:

- **Number of Layers**: How many hidden layers? (1-5)
- **Nodes per Layer**: How many neurons in each layer? (4-64)

### Tune Training Parameters:

- **Epochs**: How many training iterations? (10-300)
- **Activation Function**: Which activation to use? (relu/tanh/sigmoid)

### Experiment with Different Configurations!

- Which activation function works best?
- How does the number of epochs affect training?
- Does more layers always mean better accuracy?

### What to Observe:

- Does training accuracy improve with more epochs?
- Is there a gap between training and validation accuracy? (This indicates overfitting)
- Which configuration gives the best test accuracy?

💡 **Tip**: Try increasing layers/nodes to see if accuracy improves. But be careful - too complex models can overfit!

In [26]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# Create interactive widgets
num_layers_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=5,
    step=1,
    description='# Layers:',
    continuous_update=False
)

nodes_per_layer_slider = widgets.IntSlider(
    value=11,
    min=4,
    max=64,
    step=1,
    description='# Nodes:',
    continuous_update=False
)

epochs_slider = widgets.IntSlider(
    value=100,
    min=10,
    max=300,
    step=10,
    description='Epochs:',
    continuous_update=False
)

activation_dropdown = widgets.Dropdown(
    options=['relu', 'tanh', 'sigmoid'],
    value='relu',
    description='Activation:',
)

train_button = widgets.Button(
    description='Train Model',
    button_style='success',
    icon='check'
)

output_area = widgets.Output()

def train_model(b):
    with output_area:
        clear_output(wait=True)
        
        # Get parameters
        num_layers = num_layers_slider.value
        nodes = nodes_per_layer_slider.value
        epochs = epochs_slider.value
        activation = activation_dropdown.value
        
        print(f"Training with {num_layers} layers, {nodes} nodes per layer, {epochs} epochs, {activation} activation...")
        print("-" * 70)
        
        # Build model
        model = Sequential()
        model.add(Dense(nodes, activation=activation, input_dim=11))
        
        for i in range(num_layers - 1):
            model.add(Dense(nodes, activation=activation))
        
        model.add(Dense(1, activation='sigmoid'))  # Output layer with sigmoid for binary classification
        
        # Display model summary
        print("\nModel Architecture:")
        model.summary()
        
        # Compile model
        model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])
        
        # Train model
        print("\nTraining...")
        history = model.fit(X_train_scaled, y_train, epochs=epochs, validation_split=0.2, verbose=0)
        
        # Make predictions
        y_log = model.predict(X_test_scaled, verbose=0)
        y_pred = np.where(y_log > 0.5, 1, 0)
        
        # Calculate accuracy
        test_accuracy = accuracy_score(y_test, y_pred)
        train_accuracy = history.history['accuracy'][-1]
        val_accuracy = history.history['val_accuracy'][-1]
        
        print(f"\n{'='*70}")
        print(f"Results:")
        print(f"  Training Accuracy:   {train_accuracy:.4f}")
        print(f"  Validation Accuracy: {val_accuracy:.4f}")
        print(f"  Test Accuracy:       {test_accuracy:.4f}")
        print(f"{'='*70}")
        
        # Plot training history
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
        
        # Accuracy plot
        ax1.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
        ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
        ax1.set_xlabel('Epoch', fontsize=11)
        ax1.set_ylabel('Accuracy', fontsize=11)
        ax1.set_title('Model Accuracy', fontsize=12, fontweight='bold')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Loss plot
        ax2.plot(history.history['loss'], label='Training Loss', linewidth=2)
        ax2.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
        ax2.set_xlabel('Epoch', fontsize=11)
        ax2.set_ylabel('Loss', fontsize=11)
        ax2.set_title('Model Loss', fontsize=12, fontweight='bold')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

train_button.on_click(train_model)

# Display interface
display(widgets.VBox([
    widgets.HTML("<h3>Model Configuration</h3>"),
    num_layers_slider,
    nodes_per_layer_slider,
    epochs_slider,
    activation_dropdown,
    train_button,
    output_area
]))